In [2]:
import pandas as pd
import numpy as np

In [4]:
## onset data ## time entering a page per person
# keep column "time"
# turn "time" into array and reshape


folder_path = "/Users/apple/Desktop/deep_eye/dataset/onset_time_A/"
onset = []
for i in range(1,31):
    if i <= 9:
        path_csv = folder_path + "A0" + str(i) + ".csv"
        df = pd.read_csv(path_csv)
        onset = np.concatenate((onset, df.iloc[:47]['time']))
        #read_file.to_csv (path_csv, index = None, header=True) 
    else:
        path_csv = folder_path + "A" + str(i) + ".csv"
        df = pd.read_csv(path_csv)
        onset = np.concatenate((onset, df.iloc[:47]['time']))

print("onset len: 30*47=", len(onset))

onset_array = onset.reshape(30,47)
print(onset_array.shape)

##################################################################
## response data OK ##
# keep column "time"
# turn "time" into array and reshape

folder_path = "/Users/apple/Desktop/deep_eye/dataset/resp_time_A/"
resp = []
for i in range(1,31):
    if i <= 9:
        path_csv = folder_path + "A0" + str(i) + ".csv"
        df = pd.read_csv(path_csv)
        resp = np.concatenate((resp, df.iloc[:47]['time']))
        #read_file.to_csv (path_csv, index = None, header=True) 
    else:
        path_csv = folder_path + "A" + str(i) + ".csv"
        df = pd.read_csv(path_csv)
        resp = np.concatenate((resp, df.iloc[:47]['time']))

print("resp len: 30*47=", len(resp))

resp_array = resp.reshape(30,47)
print(resp_array.shape)

onset len: 30*47= 1410
(30, 47)
resp len: 30*47= 1410
(30, 47)


# 正式全部人

In [6]:
x_raw = pd.read_csv("/Users/apple/Desktop/deep_eye/dataset/AOI/AOI_x_2ndline.csv")
y_raw = pd.read_csv("/Users/apple/Desktop/deep_eye/dataset/AOI/AOI_y_2ndline.csv")

word_per_page_count = x_raw.iloc[-1, 1:]

x_AOI = x_raw.iloc[:-1, 1:]
y_AOI = y_raw.iloc[:-1, 1:]

All_x_AOI_range = []
for i in range(x_AOI.shape[1]):
    xpage = x_AOI.iloc[:,i]
    All_x_AOI_range.append(list(xpage[xpage.notna()]))

All_y_AOI_range = []
for i in range(y_AOI.shape[1]):
    ypage = y_AOI.iloc[:,i]
    All_y_AOI_range.append(list(ypage[ypage.notna()]))


# 文字
word_df_raw = pd.read_csv("/Users/apple/Desktop/deep_eye/dataset/AOI/AOI_text.csv").iloc[:-2,2:]
word_nona_list = []
for i in range(len(word_df_raw.values)):
    word_nona_list.append([x for x in word_df_raw.values[i] if pd.isna(x) == False])

word_df = pd.DataFrame(sum(word_nona_list, []))
word_df.columns = ["text"]
# word_df

In [9]:
####!!!!!!!! done######

origin_path = "/Users/apple/Desktop/deep_eye/dataset/raw_A/"

total_perQ_fixation_QID_df = pd.DataFrame()
for p in range(1,31):
    if p <= 9:
        path_csv = origin_path + "A0" + str(p) + ".csv"
        df = pd.read_csv(path_csv)
        df_col = df[['CURRENT_FIX_START','CURRENT_FIX_X','CURRENT_FIX_Y','CURRENT_FIX_DURATION']]
        df_col.columns = ['start_time', 'x', 'y', 'duration']
        
        # form a df with column 'start_time', 'x', 'y', and 'duration', containing each person all data
        perQ_fixation_QID_df = pd.DataFrame()
        for q in range(1,48): 
            if int(resp_array[p-1][q-1]) == 0:
                df_time = df_col[(df_col['start_time']>= int(onset_array[p-1][q-1])) & (df_col['start_time']<= int(onset_array[p-1][q-1])+14000)] # Q1,Q2...
            else:
                df_time = df_col[(df_col['start_time']>= int(onset_array[p-1][q-1])) & (df_col['start_time']<= int(resp_array[p-1][q-1]))] # Q1,Q2...
            df_time = df_time[(df_time['duration']>= 100)] # duration >= 100

            #All_fixation_total = []
            k=q-1
            All_fixation_list = []
            for i in range(len(All_x_AOI_range[k])//2):
                inrange_df = df_time[(df_time['x'] >= int(All_x_AOI_range[k][2*i])) & (df_time['x'] <= int(All_x_AOI_range[k][2*i+1])) & (df_time['y'] >= int(All_y_AOI_range[k][2*i])) & (df_time['y'] <= int(All_y_AOI_range[k][2*i+1]))]
                if inrange_df.empty:
                    All_fixation_list.append(0)
                else:
                    All_fixation_list.append(inrange_df.shape[0])
            All_fixation_total = All_fixation_list#All_fixation_total.append(All_fixation_list)
            # len(All_fixation_total) #47
            

            All_fixation_QID_df = pd.DataFrame(All_fixation_total)# df = pd.DataFrame(All_fixation_total[i])
            All_fixation_QID_df['Q_ID'] = str(q)
            All_fixation_QID_df.columns = ["fixation1", "Q_ID"]

            perQ_fixation_QID_df = pd.concat([perQ_fixation_QID_df,All_fixation_QID_df], axis = 0)
            #input_data = input_data[['Q_ID', 'text', 'fixation1']]
        #perQ_fixation_QID_df #1~47 fixation and Q_ID
        perQ_fixation_QID_df = pd.concat([perQ_fixation_QID_df.reset_index(drop=True), word_df], axis = 1)
        perQ_fixation_QID_df = perQ_fixation_QID_df[['Q_ID', 'text', 'fixation1']]
        # input_data.to_csv ("/Users/apple/Desktop/deep_eye/dataset/input_data.csv", index = None, header=True) 
      
    else:
        path_csv = origin_path + "A" + str(p) + ".csv"
        df = pd.read_csv(path_csv)
        df_col = df[['CURRENT_FIX_START','CURRENT_FIX_X','CURRENT_FIX_Y','CURRENT_FIX_DURATION']]
        df_col.columns = ['start_time', 'x', 'y', 'duration']
        
        # form a df with column 'start_time', 'x', 'y', and 'duration', containing each person all data
        perQ_fixation_QID_df = pd.DataFrame()
        for q in range(1,48): 
            if int(resp_array[p-1][q-1]) == 0:
                df_time = df_col[(df_col['start_time']>= int(onset_array[p-1][q-1])) & (df_col['start_time']<= int(onset_array[p-1][q-1])+14000)] # Q1,Q2...
            else:
                df_time = df_col[(df_col['start_time']>= int(onset_array[p-1][q-1])) & (df_col['start_time']<= int(resp_array[p-1][q-1]))] # Q1,Q2...
            df_time = df_time[(df_time['duration']>= 100)] # duration >= 100

            #All_fixation_total = []
            k=q-1
            All_fixation_list = []
            for i in range(len(All_x_AOI_range[k])//2):
                inrange_df = df_time[(df_time['x'] >= int(All_x_AOI_range[k][2*i])) & (df_time['x'] <= int(All_x_AOI_range[k][2*i+1])) & (df_time['y'] >= int(All_y_AOI_range[k][2*i])) & (df_time['y'] <= int(All_y_AOI_range[k][2*i+1]))]
                if inrange_df.empty:
                    All_fixation_list.append(0)
                else:
                    All_fixation_list.append(1)
            All_fixation_total = All_fixation_list#All_fixation_total.append(All_fixation_list)
            # len(All_fixation_total) #47
            

            All_fixation_QID_df = pd.DataFrame(All_fixation_total)# df = pd.DataFrame(All_fixation_total[i])
            All_fixation_QID_df['Q_ID'] = str(q)
            All_fixation_QID_df.columns = ["fixation1", "Q_ID"]

            perQ_fixation_QID_df = pd.concat([perQ_fixation_QID_df,All_fixation_QID_df], axis = 0)
            #input_data = input_data[['Q_ID', 'text', 'fixation1']]
        #perQ_fixation_QID_df #1~47 fixation and Q_ID
        perQ_fixation_QID_df = pd.concat([perQ_fixation_QID_df.reset_index(drop=True), word_df], axis = 1)
        perQ_fixation_QID_df = perQ_fixation_QID_df[['Q_ID', 'text', 'fixation1']]
        # input_data.to_csv ("/Users/apple/Desktop/deep_eye/dataset/input_data.csv", index = None, header=True) 
    
    perQ_fixation_QID_df["participant_ID"] = str(p)
    total_perQ_fixation_QID_df = pd.concat([total_perQ_fixation_QID_df, perQ_fixation_QID_df],axis=0)
    print(p)
total_perQ_fixation_QID_df
total_perQ_fixation_QID_df.to_csv("/Users/apple/Desktop/deep_eye/dataset/input_dataA_test.csv")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [112]:
total_perQ_fixation_QID_df

,Q_ID,text,fixation1,participant_ID
0,1,兩,0,31
1,1,個,0,31
2,1,量筒,1,31
3,1,各,0,31
4,1,裝,1,31
...,...,...,...,...
1216,47,變化,0,60
1217,47,變大,0,60
1218,47,變小,0,60
1219,47,不會,0,60


In [10]:
A_df = pd.read_csv("/Users/apple/Desktop/deep_eye/dataset/input_dataA_ver2.csv").iloc[:,1:]
C_df = pd.read_csv("/Users/apple/Desktop/deep_eye/dataset/input_dataC_ver2.csv").iloc[:,1:]
inputdata_AC = pd.concat([A_df, C_df],axis=0)
inputdata_AC = inputdata_AC[['participant_ID', 'Q_ID', 'text', 'fixation1']]
inputdata_AC.to_csv("/Users/apple/Desktop/deep_eye/dataset/input_data_ver2.csv")
inputdata_AC

,participant_ID,Q_ID,text,fixation1
0,1,1,兩,0
1,1,1,個,0
2,1,1,量筒,1
3,1,1,各,1
4,1,1,裝,0
...,...,...,...,...
36625,60,47,變化,0
36626,60,47,變大,0
36627,60,47,變小,0
36628,60,47,不會,0


---